In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io
import time
from base64 import b64encode

# Define Image Processing Functions
def enhance_sharpness(image, amount=1.0):
    """Enhance the sharpness of an image using unsharp masking."""
    blurred = ndimage.gaussian_filter(image, sigma=1)
    mask = image - blurred
    sharpened = image + amount * mask
    return np.clip(sharpened, 0, 255)

def remove_noise(image, size=3):
    """Remove noise from an image using median filtering."""
    denoised = ndimage.median_filter(image, size=size)
    return denoised

def calculate_nll(original, processed):
    """Calculate the Negative Log-Likelihood (NLL) between two images."""
    diff = processed - original
    nll = 0.5 * np.sum(diff ** 2)
    return nll

def calculate_aic(nll, k=1):
    """Calculate the Akaike Information Criterion (AIC)."""
    aic = 2 * k + 2 * nll
    return aic

# Create Widgets for User Interface
# File uploader widget
uploader = widgets.FileUpload(
    accept='image/*',  # Accept image files
    multiple=False      # Single file upload
)

# Processing options
processing_options = widgets.SelectMultiple(
    options=['Enhance Sharpness', 'Remove Noise'],
    value=[],
    description='Processing:'
)

# Buttons
process_button = widgets.Button(
    description='Process Image',
    button_style='success'
)

download_button = widgets.Button(
    description='Download Processed Image',
    button_style='info',
    disabled=True
)

# Output areas
output_image = widgets.Output()
metrics_output = widgets.Output()

# Define Event Handlers
def on_process_button_clicked(b):
    with output_image:
        clear_output()
        if not uploader.value:
            print("Please upload an image file.")
            return
        
        # Start processing timer
        start_time = time.time()
        
        # Read the uploaded image
        if isinstance(uploader.value, dict):
            # For older versions of ipywidgets where uploader.value is a dict
            uploaded_file = next(iter(uploader.value.values()))
        else:
            # For newer versions where uploader.value is a tuple/list
            uploaded_file = uploader.value[0]
        content = uploaded_file['content']
        image = Image.open(io.BytesIO(content)).convert('L')  # Convert to grayscale
        image_np = np.array(image).astype(np.float64)
        
        # Ensure image fits in memory constraints
        if image_np.nbytes > 2 * 1024 * 1024 * 1024:  # 2 GB
            print("Image is too large to process.")
            return
        
        # Keep a copy of the original image for comparison
        original_image_np = image_np.copy()
        
        # Apply selected processing options
        if 'Enhance Sharpness' in processing_options.value:
            image_np = enhance_sharpness(image_np)
        if 'Remove Noise' in processing_options.value:
            image_np = remove_noise(image_np)
        
        # Ensure processing time is under 5 seconds
        processing_time = time.time() - start_time
        if processing_time > 5:
            print("Processing took too long (>5 seconds).")
            return
        
        # Display images
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(original_image_np, cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        axes[1].imshow(image_np, cmap='gray')
        axes[1].set_title('Processed Image')
        axes[1].axis('off')
        
        plt.show()
        
        # Calculate and display metrics
        nll = calculate_nll(original_image_np, image_np)
        aic = calculate_aic(nll)
        
        with metrics_output:
            clear_output()
            print(f"Negative Log-Likelihood (NLL): {nll:.2f}")
            print(f"Akaike Information Criterion (AIC): {aic:.2f}")
        
        # Enable download button
        download_button.disabled = False
        # Save processed image in memory for download
        processed_image_pil = Image.fromarray(np.uint8(image_np))
        download_button.processed_image_pil = processed_image_pil
    
def on_download_button_clicked(b):
    processed_image_pil = b.processed_image_pil
    with io.BytesIO() as output:
        processed_image_pil.save(output, format="PNG")
        contents = output.getvalue()
        # Create a downloadable link in the notebook
        data_url = "data:image/png;base64," + b64encode(contents).decode()
        display(widgets.HTML(f'<a download="processed_image.png" href="{data_url}" target="_blank">Click here to download your image</a>'))
    
# Attach Event Handlers to Widgets
process_button.on_click(on_process_button_clicked)
download_button.on_click(on_download_button_clicked)

# Display the Interface
# Display widgets
display(widgets.VBox([
    widgets.Label("Upload an image (JPEG or PNG):"),
    uploader,
    processing_options,
    process_button,
    output_image,
    metrics_output,
    download_button
]))

